In [ ]:
import numpy as np # linear algebra
np.random.seed(666)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

In [ ]:
#Load data
train = pd.read_json("../input/train.json")
test = pd.read_json("../input/test.json")
train.inc_angle = train.inc_angle.replace('na', 0)
train.inc_angle = train.inc_angle.astype(float).fillna(0.0)
print("done!")

SVD
---

In [ ]:
rank=30


#SVD all together, find first 50 rows
X=np.array([np.array(band).astype(np.float32) for band in train["band_1"]])
Y=np.array([np.array(band).astype(np.float32) for band in test["band_1"]])
U,s,V=np.linalg.svd(np.vstack((X,Y)),full_matrices=False)
print('U,s,V',U.shape,s.shape,V.shape)
#Xr=U[:,:rank].dot(V[:rank,:])
#print('Xr',Xr.shape)

similarity matrix
---

In [ ]:
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity

total=train.set_index('id').append( test.set_index('id') )
print('total',total.shape)
#Usimil = cosine_similarity(Xr)
Usimil = cosine_similarity(U[:,:rank],V[:rank,:].T) #goes south with 10k books
print('ships+iceberg+test simil',Usimil.shape)
print(Usimil)
shix=pd.DataFrame(total.index.values,columns=['shna'])
shix['naam']=''
for xi in range(0,len(shix)):
    if xi<1604:
        if train.iat[xi,4]==0:
            shix.iat[xi,1]='b-'+str(shix.iat[xi,0])
        else:
            shix.iat[xi,1]='i-'+str(shix.iat[xi,0])
    else:
        shix.iat[xi,1]='t-'+str(shix.iat[xi,0])
print(shix.shape)
Usidf=pd.DataFrame(Usimil,index=shix['naam'])



In [ ]:
#Usidf[shix.iloc[1]['naam']].sort_values(ascending=False).head()
print(shix.iloc[1]['naam'])
Usidf[]

In [ ]:
shix.head()

In [ ]:
total['klasse']='-999'
total['prob']=0.0
total['boat']='-999'
total['boatprob']=0.0
total['is_icef']=-1.0
for xi in range(0,len(train)):
    Usi=Usidf[shix.iloc[xi]['naam']].sort_values(ascending=False)
    #Usi=Usi.index
    #print(Usi)
    ijsberg=total.iloc[xi]['is_iceberg']
    if ijsberg==1.0:
        for yi in Usi[:500].index:
            #print(yi,Usi[yi])
            if total.iat[yi,5]<Usi[yi]:
                total.iat[yi,4]=xi
                total.iat[yi,5]=Usi[yi]
                total.iat[yi,8]=1.0
    else:
        for yi in Usi[:500].index:
            #print(yi,Usi[yi])
            if total.iat[yi,7]<Usi[yi]:
                total.iat[yi,6]=xi
                total.iat[yi,7]=Usi[yi]
                total.iat[yi,8]=0.0
        
        
print( total.groupby(['klasse','is_iceberg']).count().sort_values('band_1') )
print(total.head())

In [ ]:
total['Bingo']=total['prob']>total['boatprob']
pre=total.reset_index()
pre=pre[pre.index<len(train)]
import matplotlib.pyplot as plt
plt.scatter(pre['Bingo'], train['is_iceberg'])
plt.show()

In [ ]:
total

In [ ]:
total['prob_subm']=0.0
for xi in range(0,len(total)):
    if total.iloc[xi]['prob']>total.iloc[xi]['boatprob']:
        total.iat[xi,10]=total.iloc[xi]['prob']
    else:
        total.iat[xi,10]=1-total.iloc[xi]['boatprob']


In [ ]:
submission=total[total['is_iceberg'].isnull()]['prob_subm']
submission=submission.reset_index()

submission.columns=['id','is_iceberg']
print(submission[['id','is_iceberg']].head())
submission.to_csv("./submission.csv", index=False)

In [ ]:
submission